# Embedding 模型比較

本範例展示：
1. **第1個儲存格**：使用不同 Embedding 模型建立向量資料庫
2. **第2個儲存格**：比較不同模型的檢索效果

學習目標：理解不同 Embedding 模型對檢索結果的影響，選擇最適合的模型

In [ ]:
# 第1個儲存格：使用不同 Embedding 模型建立向量資料庫

import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "健保就醫指南.txt")
db_dir = os.path.join(current_dir, "db")

# 檢查文字檔案是否存在
if not os.path.exists(file_path):
    raise FileNotFoundError(
        f"檔案 {file_path} 不存在。請檢查路徑。"
    )

# 從檔案讀取文字內容
loader = TextLoader(file_path)
documents = loader.load()

# 將文件分割成塊
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 顯示分割文件的資訊
print("\n--- 文件塊資訊 ---")
print(f"文件塊數量: {len(docs)}")
print(f"範例塊:\n{docs[0].page_content}\n")


# 建立並持久化向量存儲的函數
def create_vector_store(docs, embeddings, store_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if not os.path.exists(persistent_directory):
        print(f"\n--- 正在建立向量存儲 {store_name} ---")
        Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory)
        print(f"--- 完成建立向量存儲 {store_name} ---")
    else:
        print(
            f"向量存儲 {store_name} 已存在。無需初始化。")


# 1. Jina Embeddings v2 (繁體中文專用)
# 專門為中英雙語優化的模型
# 維度：768，最大序列長度：8192 tokens
# 適用於：繁體中文內容、長文檔處理
print("\n--- 使用 Jina Embeddings v2（繁體中文專用）---")
jina_embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-zh"
)
create_vector_store(docs, jina_embeddings, "chroma_db_jina_nb")

# 2. Multilingual E5 Large (多語言模型)
# 微軟開發的多語言嵌入模型
# 維度：1024，最大序列長度：512 tokens
# 適用於：多語言內容、高精度需求
print("\n--- 使用 Multilingual E5 Large（多語言模型）---")
e5_embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large"
)
create_vector_store(docs, e5_embeddings, "chroma_db_e5_nb")

# 3. All-MiniLM-L6-v2 (輕量級英文模型)
# Sentence Transformers 的輕量級模型
# 維度：384，最大序列長度：256 tokens
# 適用於：速度優先、資源受限環境
print("\n--- 使用 All-MiniLM-L6-v2（輕量級模型）---")
minilm_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
create_vector_store(docs, minilm_embeddings, "chroma_db_minilm_nb")

# 4. All-MPNet-Base-v2 (平衡型英文模型)
# Sentence Transformers 的平衡型模型
# 維度：768，最大序列長度：384 tokens
# 適用於：英文內容、平衡效能與速度
print("\n--- 使用 All-MPNet-Base-v2（平衡型模型）---")
mpnet_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
create_vector_store(docs, mpnet_embeddings, "chroma_db_mpnet_nb")

print("\n=== 所有向量存儲已建立完成 ===")
print("\n模型特性比較：")
print("- Jina v2: 768維，8192 tokens，繁體中文專用")
print("- E5 Large: 1024維，512 tokens，多語言高精度")
print("- MiniLM: 384維，256 tokens，輕量快速")
print("- MPNet: 768維，384 tokens，英文平衡型")

In [ ]:
# 第3個儲存格：整合 Chain 功能 - 比較不同 Embedding 模型的 RAG Chain

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 定義路徑
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 定義 Prompt 模板
prompt = ChatPromptTemplate.from_template("""你是一位專業的健保諮詢助理，請根據提供的資料回答使用者問題。

參考資料：
{context}

問題：{question}

請提供清楚、準確的回答：""")

# 格式化文件的函數
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 使用不同 Embedding 模型的 RAG Chain 並比較結果
def compare_embedding_rag_chains(store_name, query, embedding_function, model_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if os.path.exists(persistent_directory):
        print(f"\n{'='*70}")
        print(f"Embedding 模型：{model_name}")
        print(f"{'='*70}")
        
        # 載入向量資料庫
        db = Chroma(
            persist_directory=persistent_directory,
            embedding_function=embedding_function
        )
        
        # 建立檢索器
        retriever = db.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 2}
        )
        
        # 步驟 1：檢索相關文件
        print(f"\n📋 步驟 1：檢索相關文件")
        retrieved_docs = retriever.invoke(query)
        print(f"找到 {len(retrieved_docs)} 個相關文件")
        for i, doc in enumerate(retrieved_docs, 1):
            print(f"\n文件 {i} 內容預覽：")
            print(f"{doc.page_content[:150]}...")
        
        # 步驟 2：建立 RAG Chain
        print(f"\n⛓️  步驟 2：建立 RAG Chain")
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )
        
        # 步驟 3：執行 Chain 並取得答案
        print(f"\n🤖 步驟 3：生成回答")
        answer = rag_chain.invoke(query)
        print(f"\n【{model_name} 的回答】")
        print(answer)
        print(f"\n{'-'*70}")
        
        return answer
    else:
        print(f"向量存儲 {store_name} 不存在。")
        return None

# 定義不同的嵌入模型
jina_embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-zh"
)
e5_embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large"
)
minilm_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
mpnet_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# 定義使用者的問題
query = "健保卡遺失如何補辦？需要準備什麼文件？"

print("\n" + "="*70)
print(f"查詢問題：{query}")
print("="*70)

# 比較不同 Embedding 模型的 RAG Chain 效果
results = {}
results["Jina v2"] = compare_embedding_rag_chains(
    "chroma_db_jina_nb", 
    query, 
    jina_embeddings, 
    "Jina Embeddings v2 (繁體中文專用)"
)
results["E5 Large"] = compare_embedding_rag_chains(
    "chroma_db_e5_nb", 
    query, 
    e5_embeddings, 
    "Multilingual E5 Large (多語言)"
)
results["MiniLM"] = compare_embedding_rag_chains(
    "chroma_db_minilm_nb", 
    query, 
    minilm_embeddings, 
    "All-MiniLM-L6-v2 (輕量級)"
)

print("\n" + "="*70)
print("📊 Embedding 模型對 RAG Chain 的影響總結")
print("="*70)
print("""
🔍 觀察重點：
1. 不同模型檢索到的文件是否相關
2. 中文語義理解的準確度
3. 最終答案的完整性和準確性

💡 實測結果分析：
- Jina v2：對繁體中文理解最佳，檢索準確度最高
- E5 Large：多語言能力強，中文表現也不錯
- MiniLM/MPNet：主要針對英文，中文效果較差

📌 選擇建議：
✅ 繁體中文應用 → Jina Embeddings v2 (推薦！)
   - 768維向量，8192 tokens 支援長文檔
   - 專門為中英雙語優化
   
✅ 多語言應用 → Multilingual E5 Large
   - 1024維向量，100+ 語言支援
   - 適合需要處理多種語言的場景
   
✅ 英文應用 + 資源受限 → All-MiniLM-L6-v2
   - 384維向量，輕量快速
   - 適合英文為主且需要快速回應的場景

🎯 關鍵學習：
Embedding 模型的選擇直接影響檢索品質！
使用適合語言特性的模型，能大幅提升 RAG 系統效果。
""")
print("="*70)